In [1]:
import requests
import psycopg2
import socket
import json
from datetime import datetime

In [2]:

# Substitui com os teus dados reais
HOST = "db.nhbyslgfifrxsubejasw.supabase.co"
DB = "postgres"
USER = "postgres"
PASSWORD = "Database_99_!_route"  # Substitui por completo
PORT = "5432"

try:
    print("🟡 A ligar ao Supabase...")
    connection = psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
    )

    print("✅ Ligação estabelecida com sucesso!")

    cursor = connection.cursor()
    cursor.execute("SELECT NOW();")
    resultado = cursor.fetchone()
    print("🕒 Hora atual no servidor:", resultado)

    cursor.close()
    connection.close()
    print("🔒 Ligação fechada.")

except Exception as e:
    print("❌ Erro ao ligar à base de dados:", e)


🟡 A ligar ao Supabase...
✅ Ligação estabelecida com sucesso!
🕒 Hora atual no servidor: (datetime.datetime(2025, 5, 18, 17, 38, 45, 140675, tzinfo=datetime.timezone.utc),)
🔒 Ligação fechada.


In [3]:
#Connect to the server

print("🟡 A ligar ao Supabase...")
connection = psycopg2.connect(
    host=HOST,
    database=DB,
    user=USER,
    password=PASSWORD,
    port=PORT
)

print("✅ Ligação estabelecida com sucesso!")

cursor = connection.cursor()
cursor.execute("SELECT NOW();")
resultado = cursor.fetchone()
print("🕒 Hora atual no servidor:", resultado)

# ✈️ API key and airport
API_KEY = 'abcf21-5fbf11'  # Replace with your actual API key

🟡 A ligar ao Supabase...
✅ Ligação estabelecida com sucesso!
🕒 Hora atual no servidor: (datetime.datetime(2025, 5, 18, 17, 38, 45, 332268, tzinfo=datetime.timezone.utc),)


In [7]:
# 🔁 Buscar a lista de IATA dos aeroportos da Europa
def get_european_airports():
    cursor.execute('SELECT "IATA" FROM public."Airports" WHERE "IATA" IS NOT NULL;')
    return [row[0] for row in cursor.fetchall()]

# 📱 Obter voos para um aeroporto
def get_flights(flight_type, airport_code):
    url = f'https://aviation-edge.com/v2/public/timetable?key={API_KEY}&iataCode={airport_code}&type={flight_type}'
    response = requests.get(url)

    if response.status_code == 200:
        print(f"✅ {flight_type.capitalize()} flights fetched successfully for {airport_code}.")
        return response.json()
    else:
        print(f"❌ Failed to fetch {flight_type} flights for {airport_code} ({response.status_code})")
        return []

# 📂 Guardar os voos na base de dados
def save_flights(flights, flight_type):
    today = datetime.now().date()
    total = 0

    for flight in flights:
        if not isinstance(flight, dict):
            print(f"⚠️ Skipping non-dict flight record: {flight}")
            continue

        try:
            dep_info = flight.get('departure', {})
            arr_info = flight.get('arrival', {})

            dep_iata = dep_info.get('iataCode', '')
            arr_iata = arr_info.get('iataCode', '')
            flight_number = flight.get('flight', {}).get('iataNumber', '')
            dep_time = dep_info.get('scheduledTime', None)
            arr_time = arr_info.get('scheduledTime', None)

            # ❗ Check if this flight already exists
            cursor.execute('''
                SELECT 1 FROM flights
                WHERE flight_number = %s AND dep_time = %s AND arr_time = %s
            ''', (flight_number, dep_time, arr_time))

            if cursor.fetchone():
                continue  # Skip duplicates

            # Insert the flight
            cursor.execute('''
                INSERT INTO flights(
                    flight_number, airline_iata, airline_name,
                    dep_iata, dep_time, arr_iata, arr_time,
                    reg_number, status, direction, fetch_date
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', (
                flight_number,
                flight.get('airline', {}).get('iataCode', ''),
                flight.get('airline', {}).get('name', ''),
                dep_iata,
                dep_time,
                arr_iata,
                arr_time,
                flight.get('regNumber', ''),
                flight.get('status', ''),
                flight_type,
                today
            ))
            total += 1

        except Exception as e:
            flight_num = flight.get('flight', {}).get('iataNumber', 'UNKNOWN') if isinstance(flight, dict) else 'UNKNOWN'
            print(f"⚠️ Error inserting flight {flight_num}: {e}")
            connection.rollback()

    connection.commit()
    print(f"✅ {total} new {flight_type} flights saved to the database.")

# 🚀 Executar para todos os aeroportos
european_airports = get_european_airports()
total_airports = len(european_airports)

for index, airport_code in enumerate(european_airports, start=1):
    print(f"\n🌍 Processing airport {index}/{total_airports}: {airport_code}")

    departures = get_flights('departure', airport_code)
    arrivals = get_flights('arrival', airport_code)

    save_flights(departures, 'departure')
    save_flights(arrivals, 'arrival')

    print(f"✅ {index}/{total_airports} airports done\n" + "-"*40)



🌍 Processing airport 1/438: BRU
✅ Departure flights fetched successfully for BRU.
✅ Arrival flights fetched successfully for BRU.
✅ 0 new departure flights saved to the database.
✅ 0 new arrival flights saved to the database.
✅ 1/438 airports done
----------------------------------------

🌍 Processing airport 2/438: CRL
✅ Departure flights fetched successfully for CRL.
✅ Arrival flights fetched successfully for CRL.
✅ 1 new departure flights saved to the database.
✅ 0 new arrival flights saved to the database.
✅ 2/438 airports done
----------------------------------------

🌍 Processing airport 3/438: LGG
✅ Departure flights fetched successfully for LGG.
✅ Arrival flights fetched successfully for LGG.
✅ 0 new departure flights saved to the database.
✅ 0 new arrival flights saved to the database.
✅ 3/438 airports done
----------------------------------------

🌍 Processing airport 4/438: OST
✅ Departure flights fetched successfully for OST.
✅ Arrival flights fetched successfully for OST.